Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
#@title Environment Setup
import time

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull


!echo <br><br>-= Creating Python Virtual Environment =-
!pip install virtualenv
!virtualenv virtualEnvComfyUI
!source virtualEnvComfyUI/bin/activate

!echo <br><br>-= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117


In [ ]:


#@title Models To Download
modelsToDownload = {
        "SDXL": {
            "Folder": "./models/checkpoints/SDXL/",
            "Models": [
                {"url": "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors", "name": "sd_xl_base_1.0.safetensors"},
                {"url": "https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors"},
                {"url": "https://civitai.com/api/download/models/782002", "name": "Juggernaut_XL_SDXL.safetensors"}
            ]
        },
        "Pony": {
            "Folder": "./models/checkpoints/Pony/",
            "Models": [
                {"url": "https://civitai.com/api/download/models/914390", "name": "pony-realism_2.2.safetensors"},
                {"url": "https://civitai.com/api/download/models/1478064", "name": "CyberRealistic_Pony.safetensors"}
            ]
        },
        "Illustrious": {
            "Folder": "./models/checkpoints/Illustrious/",
            "Models": [
                {"url": "https://civitai.com/api/download/models/1416874", "name": "Disney_Animation_Illustrious.safetensors"},
                {"url": "https://civitai.com/api/download/models/1397168", "name": "NewGrounds_Mix_Illustrious.safetensors"},
                {"url": "https://civitai.com/api/download/models/1495132", "name": "Toonify_Illustrious.safetensors"}
            ]
        },
        "SD1.5": {
            "Folder": "./models/checkpoints/",
            "Models": [
                {"url": "https://huggingface.co/Comfy-Org/stable-diffusion-v1-5-archive/resolve/main/v1-5-pruned-emaonly-fp16.safetensors"}
            ]
        },
        "SD2": {
            "Folder": "./models/checkpoints/",
            "Models": [
                {"enabled": False, "url": "https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors"},
                {"enabled": False, "url": "https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors"}
            ]
        }
    ,
    "clip_vision": {
        "Folder": "./models/clip_vision/",
        "Models": [
            {"url": "https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors", "name": "clip_vision_g.safetensors"}
        ]
    },
    "vae": {
        "Folder": "./models/vae/",
        "Models": [
            {"url": "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors", "name": "vae-ft-mse-840000-ema-pruned.safetensors"},
            {"enabled": False, "url": "https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt"},
            {"enabled": False, "url": "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt"}
        ]
    },
    "loras": {
        "Folder": "./models/loras/",
        "Models": [
            {"url": "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors"},
            {"url": "https://civitai.com/api/download/models/10350", "name": "SD2X-theovercomer8sContrastFix_sd21768.safetensors"},
            {"enabled": False, "url": "https://civitai.com/api/download/models/10638", "name": "SD15_theovercomer8sContrastFix_sd15.safetensors"},
            {"url": "https://civitai.com/api/download/models/135867", "name": "Detail_Tweaker_SDXL.safetensors"},  # Detail Tweaker XL (SDXL) | Trigger: <lora:add-detail-xl:-1>
            {"url": "https://civitai.com/api/download/models/7657", "name": "Studio_Ghilbli_SDXL_LoRA.safetensors"},  # Studio Ghibli Style LoRA (SDXL) | Trigger: ghibli style and <lora:ghibli_style_offset:1.07>
            {"url": "https://civitai.com/api/download/models/534952", "name": "Breast_Size_Slider_Pony.safetensors"},  # Breast Size Slider - Pony | Trigger: <lora:Sketch Illustration Style [LoRA] - Pony V6 XL:0.7>
            {"url": "https://civitai.com/api/download/models/146600", "name": "breast_size_slider_SD15_LoRA.safetensors"},  # breast size slider | Trigger: <lora:breastsizeslideroffset:-1>
            {"url": "https://civitai.com/api/download/models/813349", "name": "Breasts_gravity_slider_Pony_IllustriousXL_LoRA.safetensors"},  # Breasts gravity slider Pony/IllustriousXL - LoRA | Trigger:
            {"url": "https://civitai.com/api/download/models/778093", "name": "Breasts_size_slider_Pony_Illustrious_XL_LoRA.safetensors"},  # Breasts size slider Pony/Illustrious XL | Trigger:  <lora:Breasts slider 3_last:-5>
            {"url": "https://civitai.com/api/download/models/882225", "name": "Not_Artists_Styles_Pony_SDXL.safetensors"},  # Not Artists Styles for Pony Diffusion V6 XL (SDXL) | Trigger: <lora:Sketch Illustration Style [LoRA] - Pony V6 XL:0.7>
            {"url": "https://civitai.com/api/download/models/1462485", "name": "2000s_Analog_Core_Flux_D1_LoRA.safetensors"},  # 2000s Analog Core (Flux D1) | Trigger: v8s
            {"url": "https://civitai.com/api/download/models/806265", "name": "Retro_Anime_Flux_LoRA.safetensors"},  # Retro Anime Flux - Style (Flux D1) | Trigger:
            {"url": "https://civitai.com/api/download/models/123732", "name": "Zoom_Slider_SD15_LoRA.safetensors"},  # Zoom Slider | Trigger: <lora:zoom_slider_v1:4.5>
            {"url": "https://civitai.com/api/download/models/121658", "name": "Muscle_Slider_SD15_LoRA.safetensors"},  # Muscle Slider - LoRA | Trigger: <lora:muscle_slider_v1:6>
            {"url": "https://civitai.com/api/download/models/123434", "name": "hair_length_slider_SD15_LoRA.safetensors"},  # Hair Length Slider | Trigger: <lora:hair_length_slider_v1:8>
            {"url": "https://civitai.com/api/download/models/810340", "name": "Boring_Reality_Flux-Dev_LoRA.safetensors"} , # HBoring Reality Flux-Dev LoRA | Trigger:  https://civitai.com/models/639937/boreal-fd-boring-reality-flux-dev-lora
            {"url": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15_lora.safetensors"},         # IP Adapter Face ID https://github.com/cubiq/ComfyUI_IPAdapter_plus
            {"url": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15_lora.safetensors"},  # IP Adapter Face ID https://github.com/cubiq/ComfyUI_IPAdapter_plus
            {"url": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl_lora.safetensors"},         # IP Adapter Face ID https://github.com/cubiq/ComfyUI_IPAdapter_plus
            {"url": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl_lora.safetensors"},  # IP Adapter Face ID https://github.com/cubiq/ComfyUI_IPAdapter_plus
        ]
    },
    "controlnet": {
        "Folder": "./models/controlnet/",
        "Models": [
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors"},
            {"url": "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors"},
            {"url": "https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors"}, # ControlNet SDXL
            {"url": "https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors"},
            {"url": "https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors"},
            {"url": "https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors"},
            {"enabled": False, "url": "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth"}, # T2I-Adapter
            {"enabled": False, "url": "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth"},
            {"enabled": False, "url": "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth"},
            {"enabled": False, "url": "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth"},
            {"enabled": False, "url": "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth"},
            {"enabled": False, "url": "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth"},
            {"enabled": False, "url": "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth"},
            {"url": "https://huggingface.co/InstantX/InstantID/resolve/main/ControlNetModel/diffusion_pytorch_model.safetensors", "name": "diffusion_pytorch_model_SDXL.safetensors"} # InstantFaceID
        ]
    },
    "controlnetInstandID": {
        "Folder": "./models/controlnet/instantid/",
        "Models": [
            {"url": "https://huggingface.co/InstantX/InstantID/resolve/main/ControlNetModel/diffusion_pytorch_model.safetensors", "name": "diffusion_pytorch_model_SDXL.safetensors"} # InstantFaceID
        ]
    },
    "ipadapter": {
        "Folder": "./models/ipadapter/",
        "Models": [
            {"url": "https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors"},
            {"url": "https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light_v11.bin"},
            {"url": "https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors"},
            {"url": "https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors"},
            {"url": "https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors"},
            {"url": "https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_vit-G.safetensors"},  # Requires bigG Clip vision encoder
            {"url": "https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors"},
            {"url": "https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors"},
            {"url": "https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.safetensors"},
            {"url": "https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.safetensors"},  # Requires bigG Clip vision encoder
            {"url": "https://huggingface.co/InstantX/InstantID/resolve/main/ip-adapter.bin"},                                # InstantFaceID
            {"url": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin"},                 # InstantFaceID
            {"url": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin"},          # InstantFaceID
            {"url": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait-v11_sd15.bin"},    # InstantFaceID
            {"url": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin"},                 # InstantFaceID
            {"url": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl.bin"},          # InstantFaceID
            {"url": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait_sdxl.bin"},        # InstantFaceID
            {"url": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait_sdxl_unnorm.bin"}, # InstantFaceID
        ]
    },
    "style_models": {
        "Folder": "./models/style_models/",
        "Models": [
            {"enabled": False, "url": "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth"} # T2I Styles Model
        ]
    },
    "upscale_models": {
        "Folder": "./models/upscale_models/",
        "Models": [
            {"url": "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth"}, # ESRGAN upscale model
            {"url": "https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth"},
            {"url": "https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth"}
        ]
    },
    "gligen": {
        "Folder": "./models/gligen/",
        "Models": [
            {"enabled": False, "url": "https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors"} # GLIGEN
        ]
    },
    "sams": {
        "Folder": "./models/sams/",
        "Models": [
            {"url": "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth"} # InstantFaceID
        ]
    }
}



oldCode = {"thing": """
#@title Download Models
# Checkpoints
### SDXL
### I recommend these workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/

!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P ./models/checkpoints/SDXL/
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -P ./models/checkpoints/SDXL/

!wget -c https://civitai.com/api/download/models/782002 -O ./models/checkpoints/SDXL/Juggernaut_XL_SDXL.safetensors # https://civitai.com/models/133005/juggernaut-xl


# Pony
!wget -c https://civitai.com/api/download/models/914390  -O ./models/checkpoints/Pony/pony-realism_2.2.safetensors # https://civitai.com/models/372465/pony-realism
!wget -c https://civitai.com/api/download/models/1478064 -O ./models/checkpoints/Pony/CyberRealistic_Pony.safetensors # https://civitai.com/models/443821/cyberrealistic-pony


# Illustrious
!wget -c https://civitai.com/api/download/models/1416874 -O ./models/checkpoints/Illustrious/Disney_Animation_Illustrious.safetensors # https://civitai.com/models/1256683/disney-animation-illustrious
!wget -c https://civitai.com/api/download/models/1397168 -O ./models/checkpoints/Illustrious/NewGrounds_Mix_Illustrious.safetensors # https://civitai.com/models/1239699/newgrounds-mix
!wget -c https://civitai.com/api/download/models/1495132 -O ./models/checkpoints/Illustrious/Toonify_Illustrious.safetensors # https://civitai.com/models/36281/toonify




# SDXL ReVision
!wget -c https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors -P ./models/clip_vision/

# SD1.5
!wget -c https://huggingface.co/Comfy-Org/stable-diffusion-v1-5-archive/resolve/main/v1-5-pruned-emaonly-fp16.safetensors -P ./models/checkpoints/

# SD2
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -P ./models/checkpoints/

# Some SD1.5 anime style
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors -P ./models/checkpoints/

# Waifu Diffusion 1.5 (anime style SD2.x 768-v)
#!wget -c https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors -P ./models/checkpoints/


# unCLIP models
#!wget -c https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/resolve/main/illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/comfyanonymous/wd-1.5-beta2_unCLIP/resolve/main/wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors -P ./models/checkpoints/


# VAE
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -P ./models/vae/
#!wget -c https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -P ./models/vae/


## Loras
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors -P ./models/loras/ #SDXL offset noise lora
!wget -c https://civitai.com/api/download/models/10350 -O ./models/loras/SD2X-theovercomer8sContrastFix_sd21768.safetensors #theovercomer8sContrastFix SD2.x 768-v
!wget -c https://civitai.com/api/download/models/10638 -O ./models/loras/SD15_theovercomer8sContrastFix_sd15.safetensors #theovercomer8sContrastFix SD1.x


!wget -c https://civitai.com/api/download/models/135867 -O ./models/loras/Detail_Tweaker_SDXL.safetensors # Detail Tweaker XL (SDXL) | Trigger: <lora:add-detail-xl:-1>
!wget -c https://civitai.com/api/download/models/7657 -O ./models/loras/Studio_Ghilbli_SDXL_LoRA.safetensors # Studio Ghibli Style LoRA (SDXL) | Trigger: ghibli style and <lora:ghibli_style_offset:1.07>


!wget -c https://civitai.com/api/download/models/534952 -O ./models/loras/Breast_Size_Slider_Pony.safetensors # Breast Size Slider - Pony | Trigger: <lora:Sketch Illustration Style [LoRA] - Pony V6 XL:0.7>
!wget -c https://civitai.com/api/download/models/146600 -O ./models/loras/breast_size_slider_SD15_LoRA.safetensors # breast size slider | Trigger: <lora:breastsizeslideroffset:-1>
!wget -c https://civitai.com/api/download/models/813349 -O ./models/loras/Breasts_gravity_slider_Pony_IllustriousXL_LoRA.safetensors # Breasts gravity slider Pony/IllustriousXL - LoRA | Trigger:
!wget -c https://civitai.com/api/download/models/778093 -O ./models/loras/Breasts_size_slider_Pony_Illustrious_XL_LoRA.safetensors # Breasts size slider Pony/Illustrious XL | Trigger:  <lora:Breasts slider 3_last:-5>

!wget -c https://civitai.com/api/download/models/882225 -O ./models/loras/Not_Artists_Styles_Pony_SDXL.safetensors # Not Artists Styles for Pony Diffusion V6 XL (SDXL) | Trigger: <lora:Sketch Illustration Style [LoRA] - Pony V6 XL:0.7>
!wget -c https://civitai.com/api/download/models/1462485 -O ./models/loras/2000s_Analog_Core_Flux_D1_LoRA.safetensors # 2000s Analog Core (Flux D1) | Trigger: v8s
!wget -c https://civitai.com/api/download/models/806265 -O ./models/loras/Retro_Anime_Flux_LoRA.safetensors # Retro Anime Flux - Style (Flux D1) | Trigger:
!wget -c https://civitai.com/api/download/models/123732 -O ./models/loras/Zoom_Slider_SD15_LoRA.safetensors # Zoom Slider | Trigger: <lora:zoom_slider_v1:4.5>
!wget -c https://civitai.com/api/download/models/121658 -O ./models/loras/Muscle_Slider_SD15_LoRA.safetensors # Muscle Slider - LoRA | Trigger: <lora:muscle_slider_v1:6>

!wget -c https://civitai.com/api/download/models/123434 -O ./models/loras/hair_length_slider_SD15_LoRA.safetensors # Hair Length Slider | Trigger: <lora:hair_length_slider_v1:8>
!wget -c https://civitai.com/api/download/models/810340 -O ./models/loras/Boring_Reality_Flux-Dev_LoRA.safetensors # HBoring Reality Flux-Dev LoRA | Trigger:  https://civitai.com/models/639937/boreal-fd-boring-reality-flux-dev-lora




# T2I-Adapter
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -P ./models/controlnet/

# T2I Styles Model
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P ./models/style_models/

# CLIPVision model (needed for styles model)
#!wget -c https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -O ./models/clip_vision/clip_vit14.bin


# IP Adapters
## ClipVision IP Adapters
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -O ./models/clip_vision/CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors -O ./models/clip_vision/CLIP-ViT-bigG-14-laion2B-39B-b160k.safetensors
!wget -c https://huggingface.co/Kwai-Kolors/Kolors-IP-Adapter-Plus/resolve/main/image_encoder/pytorch_model.bin -O ./models/clip_vision/clip-vit-large-patch14-336.bin

## IP Adapter
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors -P ./models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light_v11.bin -P ./models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors -P ./models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors -P ./models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors -P ./models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_vit-G.safetensors -P ./models/ipadapter/     # Requires bigG Clip vision encoder
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors -P ./models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors -P ./models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.safetensors -P ./models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.safetensors -P ./models/ipadapter/      # Requires bigG Clip vision encoder


# InstantFaceID
!wget -c https://huggingface.co/InstantX/InstantID/resolve/main/ip-adapter.bin -P ./models/instantid/
!wget -c https://huggingface.co/InstantX/InstantID/resolve/main/ControlNetModel/diffusion_pytorch_model.safetensors -O ./models/controlnet/instantid/diffusion_pytorch_model_SDXL.safetensors
!wget -c https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P ./models/sams/


# ControlNet
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -P ./models/controlnet/

# ControlNet SDXL
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors -P ./models/controlnet/

# Controlnet Preprocessor nodes by Fannovel16
#!cd custom_nodes && git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors; cd comfy_controlnet_preprocessors && python install.py


# GLIGEN
#!wget -c https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -P ./models/gligen/


# ESRGAN upscale model
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P ./models/upscale_models/
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P ./models/upscale_models/
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P ./models/upscale_models/
"""}

In [ ]:
#@title Download models

failedDownloads = []
if 'modelsToDownload' not in locals():
  raise NameError("Can't find variable 'models'. Most likely you need to run the 'Models To Download' cell")
for category, details in modelsToDownload.items():
    time.sleep(1)
    # for subcategory, details in subcategories.items():
    folder = details.get("Folder")  # Use get() to avoid KeyError
    if folder is None:
        continue  # Skip if "Folder" key is not found
    for model in details.get("Models", []):  # Use get() with default value
        if "enabled" in model and not model["enabled"]:
            continue  # Skip disabled downloads
        url = model["url"]
        filename = model.get("name", url.split("/")[-1])
        print(f"Downloading {filename}...")
        try:
            !wget -c {url} -O {folder}{filename}
        except subprocess.CalledProcessError as e:
          if e.returncode != 0: # Check if the command failed
            print(f"Failed to download {filename}. Status code: {e.returncode}")
            failedDownloads.append([filename, e.returncode])  # Add failed download to list
# Print the names of failed downloads
if modelsToDownload:
    print("\nFailed to download the following models:")
    for model_name in failedDownloads:
        print(f"Name: {model_name[0]}, {model_name[1]}")

In [ ]:
#@title Install custom nodes
%cd $WORKSPACE
CUSTOMNODES = "/content/drive/MyDrive/ComfyUI/custom_nodes"
!git config pull.rebase false --global

%cd $CUSTOMNODES
!echo <br>

!git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack
%cd ./ComfyUI-Impact-Pack
!pip install -r requirements.txt

%cd $CUSTOMNODES
!echo <br>

!git clone https://github.com/cubiq/ComfyUI_InstantID
%cd ./ComfyUI_InstantID
!pip install -r requirements.txt

%cd $CUSTOMNODES
!echo <br>

!git clone https://github.com/ltdrdata/ComfyUI-Impact-Subpack
%cd ./ComfyUI-Impact-Subpack
!pip install -r requirements.txt

%cd $CUSTOMNODES
!echo <br>
!git clone https://github.com/ltdrdata/comfyui_controlnet_aux
%cd ./comfyui_controlnet_aux
!pip install -r requirements.txt
!echo <br>
!pip install simpleeval

%cd $WORKSPACE

### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

### Run ComfyUI with localtunnel




In [ ]:
!npm install -g localtunnel

import threading

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

### Run ComfyUI with colab iframe (use only in case the previous way with localtunnel doesn't work)

You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.

If you want to open it in another window use the link.

Note that some UI features like live image previews won't work because the colab iframe blocks websockets.

In [ ]:
import threading
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server